In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
import model as m
from data_layer import DATA,Copula,RealData
import numpy as np
from keras import backend as K
from metric import Test,copula_cmp
import numpy.linalg
import loss
import sys
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%config InlineBackend.figure_format = 'retina'
plt.rc('figure', figsize=(3, 3))

# New Section

In [7]:
filepath = '/content/drive/My Drive/Data/gtex6/gtex-skin-wholeblood/original/Data/'# this is directory path #sys.argv[1]
pca_com =  50 # How many pca components you want to take #int(sys.argv[2])
copula_num = 4000 # Number of samples generated by Copula #int(sys.argv[3])
activation = 'selu'# Nonlinearity in Model #sys.argv[4]
activation_last_layer = 'linear'# activation of last layer #sys.argv[4]
train_op = 1 # Type of training int(sys.argv[6])
test_op = 1 # 1 to predict test left to right generation, 2 to right to left and 3 for both  #int(sys.argv[7])
lamda = 0.1  # A constant for CorNet to balance Correlation loss #float(sys.argv[8])
dpca = 25

data = RealData(filepath,pca_com)

lc,rc,lu,ru = data.get_data()

#np.random.shuffle(lc)
#np.random.shuffle(rc)

num = len(lc)
tnum = int(num*0.7) # 70-30 train test split
test = int(num*0.3)
print("training", tnum," testing", test)

model_dim = len(lc[0])

print(num,model_dim)

models = m.Models(model_dim, model_dim, int(model_dim/0.1), int(model_dim/0.2), int(model_dim/0.5), activation,activation_last_layer, lamda)
#models = m.Models(model_dim, model_dim, dpca, int(dpca/1.8), int(dpca/2.5), activation,activation_last_layer, lamda)
model, sml, smr = models.getModel()
if(copula_num>0 and pca_com != -1):
    d = np.concatenate([lc[:tnum],rc[:tnum]],axis=1)
    cop = Copula(d)
    sim_data = cop.gendata(copula_num)
    sim_data = np.array(sim_data)
    l_sim = sim_data[:,0:model_dim]
    r_sim = sim_data[:,model_dim:]
    left = np.concatenate([lc[:tnum],l_sim],axis=0)
    right = np.concatenate([rc[:tnum],r_sim],axis=0)
    left = left.tolist()
    right = right.tolist()
elif(copula_num>0 and pca_com == -1):
    data1 = RealData(filepath,dpca)
    lc1,rc1,lu1,ru1 = data1.get_data()
    d = np.concatenate([lc1[:tnum],rc1[:tnum]],axis=1)
    cop = Copula(d)
    sim_data = cop.gendata(copula_num)
    sim_data = np.array(sim_data)
    l_sim = sim_data[:,0:dpca]
    r_sim = sim_data[:,dpca:]
    print("if not great problem")
    l_sim = data1.Transform.inv_trans_left(l_sim)
    r_sim = data1.Transform.inv_trans_right(r_sim)
    left = np.concatenate([lc[:tnum],l_sim],axis=0)
    right = np.concatenate([rc[:tnum],r_sim],axis=0)
    left = left.tolist()
    right = right.tolist()

else:
    left = lc[:tnum]
    right = rc[:tnum]

train_val_split = int(len(left)*0.7)
val_left = left[train_val_split:]
val_right = right[train_val_split:]

left = left[:train_val_split] 
right = right[:train_val_split]

test_left = lc[tnum:]
test_right = rc[tnum:]
print(len(val_left), len(val_left[0]))
print(len(test_left), len(test_left[0]))




training 181  testing 77
259 50
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 500)          25500       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 500)          25500       input_2[0][0]                    
_______________________________________________________

In [8]:
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
import loss   

In [ ]:

X_train_l = np.array(left)
X_train_r = np.array(right)
val_left = np.array(val_left)
val_right = np.array(val_right)
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4, min_lr=0.00000001)
model.fit([X_train_l,X_train_r], [X_train_l,X_train_l,X_train_l,X_train_r,X_train_r,X_train_r,np.ones((X_train_l.shape[0],X_train_l.shape[1]))],
            epochs=800,batch_size=30,verbose=0,
            validation_data=([val_left,val_right],[val_left,val_left,val_left,val_right,val_right,val_right,np.ones((val_left.shape[0],val_left.shape[1]))]),
            shuffle=False,
              callbacks=[
              checkpointer,reduce_lr
          ]
            
            )
model.load_weights('model.h5')




ru_=[]
lu_=[]

cov_mat=None
#fig, ax = plt.subplots(figsize=(50, 50),nrows=1, ncols=1)
#ax.imshow(cov_mat, cmap='binary', interpolation='nearest')
#fig.savefig('copula_test.png')



Epoch 00001: val_loss improved from inf to 630.61542, saving model to model.h5

Epoch 00002: val_loss improved from 630.61542 to 550.33484, saving model to model.h5

Epoch 00003: val_loss improved from 550.33484 to 524.69952, saving model to model.h5

Epoch 00004: val_loss improved from 524.69952 to 521.31445, saving model to model.h5

Epoch 00005: val_loss improved from 521.31445 to 518.55310, saving model to model.h5

Epoch 00006: val_loss did not improve from 518.55310

Epoch 00007: val_loss did not improve from 518.55310

Epoch 00008: val_loss did not improve from 518.55310

Epoch 00009: val_loss did not improve from 518.55310

Epoch 00010: val_loss did not improve from 518.55310

Epoch 00011: val_loss did not improve from 518.55310

Epoch 00012: val_loss did not improve from 518.55310

Epoch 00013: val_loss did not improve from 518.55310

Epoch 00014: val_loss did not improve from 518.55310

Epoch 00015: val_loss did not improve from 518.55310

Epoch 00016: val_loss improved from

In [ ]:
test = Test(data.Transform,data.Cleft[tnum:],data.Cright[tnum:],test_left,test_right,lu,ru_,lu_,ru,cov_mat,2, model)
if True:
    test.test_LtoR()
    test.test_RtoL()

In [ ]:
del test
!pip install shap

In [ ]:
!pip install pyvis

In [ ]:
ar = data.Transform.pca_l.components_[0]
print(data.Transform.pca_l.components_.shape)
ll = np.argsort(ar)
a = ll[18700:]
print(data.left_gene[a])

In [ ]:
import shap
from sklearn.preprocessing import normalize
from pyvis.network import Network
import pickle

class Importance:
  def __init__(self, right, left,sml,smr, test_left, test_right, data):
    shap_right = np.array(right)
    shap_left = np.array(left)
    self.background_r = [np.zeros_like(shap_left), shap_right]
    self.background_l = [shap_left, np.zeros_like(shap_right)]

    s_test_right = np.array(test_right)
    s_test_left = np.array(test_left)
    self.background_t_l = [s_test_left,np.zeros_like(s_test_right)]
    self.background_t_r = [np.zeros_like(s_test_left),s_test_right]

    self.data = data
    self.sml = sml
    self.smr = smr
    self.gene_to_gene_right_for_left = None
    self.gene_to_gene_left_for_left = None
    self.gene_to_gene_left_for_right = None
    self.gene_to_gene_right_for_right = None

    self.important_indexs_right_for_left = None
    self.important_indexs_left_for_left = None
    self.important_indexs_left_for_right = None
    self.important_indexs_right_for_right = None

    self.g2g_l2l  = None
    self.g2g_r2l = None
    self.g2g_r2r = None
    self.g2g_l2r = None

  def calculate_np(self,):

    e = shap.GradientExplainer(self.sml, self.background_r)
    shap_values = e.shap_values(self.background_t_r)
    path = '/content/drive/My Drive/skin-WBC'

    #extracting importance of right input and porjecting to gene space
    self.gene_to_gene_right_for_left = [i[1].mean(axis=0).tolist() for i in shap_values]
    #pick top 3 
    self.important_indexs_right_for_left = []
    for i in self.gene_to_gene_right_for_left:
      a = np.argsort(i)[-3:]
      self.important_indexs_right_for_left.append(a)
    #saving data into path
    '''
    with open(path+'/left-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)

    del self.gene_to_gene_right_for_left'''
    del e
    e = shap.GradientExplainer(self.sml, self.background_l)
    shap_values = e.shap_values(self.background_t_l)

    #extracting importance of right input and porjecting to gene space
    self.gene_to_gene_left_for_left = [i[0].mean(axis=0).tolist() for i in shap_values]
    #pick top 3 
    self.important_indexs_left_for_left = []
    for i in self.gene_to_gene_left_for_left:
      a = np.argsort(i)[-3:]
      self.important_indexs_left_for_left.append(a)

    path = '/content/drive/My Drive/skin-WBC'
    print('done 1')
    del e
    e = shap.GradientExplainer(self.smr, self.background_l)
    shap_values = e.shap_values(self.background_t_l)

    #extracting importance of right input and porjecting to gene space
    self.gene_to_gene_left_for_right = [i[0].mean(axis=0).tolist() for i in shap_values]
    
    #self.gene_to_gene_left_for_right = np.abs(self.gene_to_gene_left_for_right)
    #pick top 3 
    self.important_indexs_left_for_right = []
    for i in self.gene_to_gene_left_for_right:
      a = np.argsort(i)[-3:]
      self.important_indexs_left_for_right.append(a)
    '''
    with open(path+'/right-left.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_left_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
    
    del e
    e = shap.GradientExplainer(self.smr, self.background_r)
    shap_values = e.shap_values(self.background_t_r)

    #extracting importance of right input and porjecting to gene space
    self.gene_to_gene_right_for_rightt = [i[1].mean(axis=0).tolist() for i in shap_values]
    #self.gene_to_gene_right_for_right = np.abs(self.gene_to_gene_right_for_right)
    #pick top 3 
    self.important_indexs_right_for_right = []
    for i in self.gene_to_gene_right_for_right:
      a = np.argsort(i)[-3:]
      self.important_indexs_right_for_right.append(a)
    '''
    with open(path+'/right-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
   

  def calculate_l(self,):

    e = shap.GradientExplainer(self.sml, self.background_r)
    shap_values = e.shap_values(self.background_t_r)
    path = '/content/drive/My Drive/skin-WBC'

    #extracting importance of right input and porjecting to gene space
    right_importance_for_left = [i[1].mean(axis=0).tolist() for i in shap_values]
    right_importance_gene_for_left = self.data.Transform.pca_r.inverse_transform(right_importance_for_left)
    left_eigen_evctors = self.data.Transform.pca_l.components_

    #From (pca, gene) to (gene, gene) space
    left_eigen_evctors = np.array(left_eigen_evctors)
    left_eigen_evctors = left_eigen_evctors.T
    right_importance_gene_for_left = np.array(right_importance_gene_for_left)
    self.gene_to_gene_right_for_left = left_eigen_evctors.dot(right_importance_gene_for_left)
    self.gene_to_gene_right_for_left = self.data.Transform.std_l.var_ * self.gene_to_gene_right_for_left
    
    
    

    #pick top 3 
    self.important_indexs_right_for_left = []
    for i in self.gene_to_gene_right_for_left:
      a = np.argsort(i)[-3:]
      self.important_indexs_right_for_left.append(a)
    #saving data into path
    '''
    with open(path+'/left-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)

    del self.gene_to_gene_right_for_left'''
    del e
    e = shap.GradientExplainer(self.sml, self.background_l)
    shap_values = e.shap_values(self.background_t_l)

    #extracting importance of right input and porjecting to gene space
    left_importance_for_left = [i[0].mean(axis=0).tolist() for i in shap_values]
    #left_importance_for_left = normalize(left_importance_for_left, norm = 'l2' , axis=0)
    left_importance_gene_for_left = self.data.Transform.pca_l.inverse_transform(left_importance_for_left)
    left_eigen_evctors = self.data.Transform.pca_l.components_

    #From (pca, gene) to (gene, gene) space
    left_eigen_evctors = np.array(left_eigen_evctors)
    left_eigen_evctors = left_eigen_evctors.T
    left_importance_gene_for_left = np.array(left_importance_gene_for_left)
    self.gene_to_gene_left_for_left = left_eigen_evctors.dot(left_importance_gene_for_left)
    self.gene_to_gene_left_for_left = self.data.Transform.std_l.var_ * (self.gene_to_gene_left_for_left)
    #pick top 3 
    self.important_indexs_left_for_left = []
    for i in self.gene_to_gene_left_for_left:
      a = np.argsort(i)[-3:]
      self.important_indexs_left_for_left.append(a)
    '''
    with open(path+'/left-left.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_left_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
    
  
  def calculate_r(self,):
    path = '/content/drive/My Drive/skin-WBC'
    e = shap.GradientExplainer(self.smr, self.background_l)
    shap_values = e.shap_values(self.background_t_l)

    #extracting importance of right input and porjecting to gene space
    left_importance_for_right = [i[0].mean(axis=0).tolist() for i in shap_values]
    left_importance_gene_for_right = self.data.Transform.pca_l.inverse_transform(left_importance_for_right)
    left_eigen_evctors = self.data.Transform.pca_r.components_

    #From (pca, gene) to (gene, gene) space
    left_eigen_evctors = np.array(left_eigen_evctors)
    left_eigen_evctors = left_eigen_evctors.T
    left_importance_gene_for_right = np.array(left_importance_gene_for_right)
    self.gene_to_gene_left_for_right = left_eigen_evctors.dot(left_importance_gene_for_right)
    self.gene_to_gene_left_for_right = self.data.Transform.std_r.var_ * (self.gene_to_gene_left_for_right)
    #pick top 3 
    self.important_indexs_left_for_right = []
    for i in self.gene_to_gene_left_for_right:
      a = np.argsort(i)[-3:]
      self.important_indexs_left_for_right.append(a)
    '''
    with open(path+'/right-left.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_left_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)'''

    del e
    e = shap.GradientExplainer(self.smr, self.background_r)
    shap_values = e.shap_values(self.background_t_r)

    #extracting importance of right input and porjecting to gene space
    right_importance_for_right = [i[1].mean(axis=0).tolist() for i in shap_values]
    #right_importance_for_right = normalize(right_importance_for_right, norm = 'l2' , axis=0)
    right_importance_gene_for_right = self.data.Transform.pca_r.inverse_transform(right_importance_for_right)
    left_eigen_evctors = self.data.Transform.pca_r.components_

    #From (pca, gene) to (gene, gene) space
    left_eigen_evctors = np.array(left_eigen_evctors)
    left_eigen_evctors = left_eigen_evctors.T
    right_importance_gene_for_right = np.array(right_importance_gene_for_right)
    self.gene_to_gene_right_for_right = left_eigen_evctors.dot(right_importance_gene_for_right)
    self.gene_to_gene_right_for_right = self.data.Transform.std_r.var_ * (self.gene_to_gene_right_for_right)
    #pick top 3 
    self.important_indexs_right_for_right = []
    for i in self.gene_to_gene_right_for_right:
      a = np.argsort(i)[-3:]
      self.important_indexs_right_for_right.append(a)
    '''
    with open(path+'/right-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
    
  
  def get_gradient(self):
    self.g2g_l2l = np.sum( self.gene_to_gene_left_for_left, axis=0 )
    self.g2g_r2r = np.sum( self.gene_to_gene_right_for_right, axis=0)

    self.g2g_r2l = np.sum( self.gene_to_gene_left_for_right, axis=0) 
    self.g2g_l2r = np.sum( self.gene_to_gene_right_for_left, axis=0 )
  
  def getTop_g(self,):
    return np.argsort(self.g2g_l2l), np.argsort(self.g2g_r2l), np.argsort(self.g2g_r2r), np.argsort(self.g2g_l2r)
  
  def getTop_v(self,V):
    if V== -1:
      return np.nonzero(np.average(self.data.Cleft,axis=0)), np.nonzero(np.average(self.data.Cright,axis=0))

    return np.argsort(np.average(self.data.Cleft,axis=0))[-1*V:], np.argsort(np.average(self.data.Cright,axis=0))[-1*V:]
  
  def build_graph_1(self,num):
    idex_l = np.argsort(self.g2g_l2l)
    idex_r = np.argsort(self.g2g_r2r)
    idex_rl = np.argsort(self.g2g_r2l)
    idex_lr = np.argsort(self.g2g_l2r)

    net = Network(height="1200px", width="100%", bgcolor="white", font_color="black")
    net.barnes_hut()
    net.show_buttons(filter_=['physics'])
    color = []

    num = -1*num
    #right - right connecting - edge color green
    for i in idex_r[num:]: #random.sample(range(len(important_indexs_right_for_right)), k=15):
      n1 = self.data.right_gene[i]
      
      color.append('red')  
      n2,n3,n4 = self.data.right_gene[importance.important_indexs_right_for_right[i]]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_nodes([n3], color=['green'], size=[5])
      net.add_nodes([n4], color=['green'], size=[5])
      net.add_edge(n1,n2,color='springgreen')
      net.add_edge(n1,n3,color='springgreen')
      net.add_edge(n1,n4,color='springgreen')

    #left to left connecting - edge color blue
    for i in idex_l[num:]: #random.sample(range(len(important_indexs_left_for_left)), k=15):
      n1 = self.data.left_gene[i]
      
      color.append('red')  
      n2,n3,n4 = self.data.left_gene[importance.important_indexs_left_for_left[i]]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_nodes([n3], color=['green'], size=[5])
      net.add_nodes([n4], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#459ced')
      net.add_edge(n1,n3,color='#459ced')
      net.add_edge(n1,n4,color='#459ced')

    for i in idex_rl[num:]: #random.sample(range(len(important_indexs_right_for_left)), k=15):
      n1 = self.data.left_gene[i]
      
      color.append('red')  
      n2,n3,n4 = self.data.right_gene[importance.important_indexs_right_for_left[i]]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_nodes([n3], color=['green'], size=[5])
      net.add_nodes([n4], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#e85815')
      net.add_edge(n1,n3,color='#e85815')
      net.add_edge(n1,n4,color='#e85815')

    for i in idex_lr[num:]: #random.sample(range(len(important_indexs_left_for_right)), k=15):
      n1 = self.data.right_gene[i]
      
      color.append('red')  
      n2,n3,n4 = self.data.left_gene[importance.important_indexs_left_for_right[i]]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_nodes([n3], color=['green'], size=[5])
      net.add_nodes([n4], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#e85815')
      net.add_edge(n1,n3,color='#e85815')
      net.add_edge(n1,n4,color='#e85815')
    
    net.show("Gene2gene_1.html")
  def save_data(self,path):
    import pickle

    with open(path+'/left-left.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_left_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(path+'/right-left.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_left_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(path+'/right-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open(path+'/left-right.pickle', 'wb') as handle:
        pickle.dump(self.gene_to_gene_right_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(path+'/left-names.pickle', 'wb') as handle:
        pickle.dump(self.data.left_gene, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open(path+'/right-names.pickle', 'wb') as handle:
        pickle.dump(self.data.right_gene, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


  def build_graph_2(self, num):
    net = Network(height="1200px", width="100%", bgcolor="white", font_color="black")
    net.barnes_hut()
    net.show_buttons(filter_=['physics'])
    color = []

    num = -1*num
    self.calculate_l()
    #left to left connecting - edge color blue
    ii = np.unravel_index(np.argpartition(self.gene_to_gene_left_for_left.ravel(),num)[num:], self.gene_to_gene_left_for_left.shape)
    g1 = ii[0]
    g2 = ii[1]
    for i in range(len(g1)): #random.sample(range(len(important_indexs_right_for_right)), k=15):
      j = g1[i]
      k = g2[i]
      n1 = self.data.left_gene[j]  
      n2 = self.data.left_gene[k]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#459ced')

    #left - right
    ii = np.unravel_index(np.argpartition(self.gene_to_gene_right_for_left.ravel(),num)[num:], self.gene_to_gene_right_for_left.shape)
    g1 = ii[0]
    g2 = ii[1]
    for i in range(len(g1)): #random.sample(range(len(important_indexs_right_for_right)), k=15):
      j = g1[i]
      k = g2[i]
      n1 = self.data.left_gene[j]  
      n2 = self.data.right_gene[k]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#e85815')

    del self.gene_to_gene_right_for_left
    del self.gene_to_gene_left_for_left

    self.calculate_r()
    #right left
    ii = np.unravel_index(np.argpartition(self.gene_to_gene_left_for_right.ravel(),num)[num:], self.gene_to_gene_left_for_right.shape)
    g1 = ii[0]
    g2 = ii[1]
    for i in range(len(g1)): #random.sample(range(len(important_indexs_right_for_right)), k=15):
      j = g1[i]
      k = g2[i]
      n1 = self.data.right_gene[j]  
      n2 = self.data.left_gene[k]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_edge(n1,n2,color='#e85815')

    #right - right connecting - edge color green
    ii = np.unravel_index(np.argpartition(self.gene_to_gene_right_for_right.ravel(),num)[num:], self.gene_to_gene_right_for_right.shape)
    g1 = ii[0]
    g2 = ii[1]
    for i in range(len(g1)): #random.sample(range(len(important_indexs_right_for_right)), k=15):
      j = g1[i]
      k = g2[i]
      n1 = self.data.right_gene[j]  
      n2 = self.data.right_gene[k]
      net.add_nodes([n1], color=['green'], size=[5])
      net.add_nodes([n2], color=['green'], size=[5])
      net.add_edge(n1,n2,color='springgreen')

    net.show("gene2gene_DS.html")




In [ ]:
importance = Importance(right,left,sml,smr,lc,rc,data)
importance.build_graph_2(300)
del importance

In [ ]:

importance = Importance(right,left,sml,smr,lc,rc,data)
importance.calculate_l()


In [ ]:
importance.calculate_r()

In [ ]:
importance.get_gradient()

In [ ]:
def getrank(argso):
  l = [0 for i in argso]
  for i in range(len(argso)):
    l[argso[i]]=i
  return l

In [ ]:
ll = np.argsort(importance.g2g_l2l)
a = ll[-1]
print(data.left_gene[ll[-1]])
ll = getrank(ll)
print(ll[a], len(ll))

KRT10
18718 18719


In [ ]:
rl = np.argsort(importance.g2g_r2l)
print(data.left_gene[rl[-1]])
rl = getrank(rl)

rr = np.argsort(importance.g2g_r2r)
print(data.right_gene[rr[-1]])
rr = getrank(rr)

lr = np.argsort(importance.g2g_l2r)
print(data.right_gene[lr[-1]])
lr = getrank(lr)

HBB
S100A9
KRT10


In [ ]:
lv, rv = np.argsort(np.average(data.Cleft,axis=0)), np.argsort(np.average(data.Cright,axis=0))

In [ ]:
print(data.left_gene[lv[-1]])
lv = getrank(lv)
print(data.right_gene[rv[-1]])
rv = getrank(rv)

KRT10
HBB


In [ ]:
from scipy.stats import spearmanr
l1 = spearmanr(ll,rv)
l2 = spearmanr(rl,rv)
r1= spearmanr(rr,lv)
r2 = spearmanr(lr,lv)
print(l1,l2,r1,r2)

SpearmanrResult(correlation=-0.16272864922167796, pvalue=2.9956054901140355e-111) SpearmanrResult(correlation=-0.018050012903058493, pvalue=0.013526722044313638) SpearmanrResult(correlation=0.6043251345640764, pvalue=0.0) SpearmanrResult(correlation=-0.14148675837155716, pvalue=2.6697099915840024e-84)


In [ ]:
print(spearmanr(ll,rl), spearmanr(rr,lr), spearmanr(ll,rr))



SpearmanrResult(correlation=0.1789744444919525, pvalue=1.55998280307853e-134) SpearmanrResult(correlation=-0.1874448545697496, pvalue=1.3186424651639537e-147) SpearmanrResult(correlation=-0.16590297197115925, pvalue=1.290058898551451e-115)


In [ ]:
lx = np.argsort(importance.g2g_r2r)
rl = np.argsort(importance.g2g_l2r)
name_l = data.right_gene[lx]
name_rl = data.right_gene[rl]

temp = open('rankr.txt','w')
for i in reversed(name_l):
  temp.write(i+'\n')
temp.close()

temp = open('ranklr.txt','w')
for i in reversed(name_rl):
  temp.write(i+'\n')
temp.close()


In [ ]:
lx = np.argsort(importance.g2g_l2l)
rl = np.argsort(importance.g2g_r2l)
name_l = data.left_gene[lx]
name_rl = data.left_gene[rl]

temp = open('rankl.txt','w')
for i in reversed(name_l):
  temp.write(i+'\n')
temp.close()

temp = open('rankrl.txt','w')
for i in reversed(name_rl):
  temp.write(i+'\n')
temp.close()



In [ ]:
lv, rv = np.argsort(np.average(data.Cleft,axis=0)), np.argsort(np.average(data.Cright,axis=0))
name_l = data.left_gene[lv]
name_rl = data.left_gene[rv]

temp = open('expl.txt','w')
for i in reversed(name_l):
  temp.write(i+'\n')
temp.close()

temp = open('expr.txt','w')
for i in reversed(name_rl):
  temp.write(i+'\n')
temp.close()

In [ ]:
importance.build_graph_1(50)

In [ ]:
with open(path+'/right-left.pickle', 'wb') as handle:
    pickle.dump(self.gene_to_gene_left_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+'/right-right.pickle', 'wb') as handle:
    pickle.dump(self.gene_to_gene_right_for_right, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+'/left-right.pickle', 'wb') as handle:
    pickle.dump(self.gene_to_gene_right_for_left, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+'/left-names.pickle', 'wb') as handle:
    pickle.dump(self.data.left_gene, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+'/right-names.pickle', 'wb') as handle:
    pickle.dump(self.data.right_gene, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
'''plt.boxplot(sum_g2g_l2l,patch_artist=True,autorange= True, showfliers=False,vert=True)
plt.xlabel("IMPORTANCE lung")
plt.show()
plt.boxplot(sum_g2g_r2r,patch_artist=True,autorange = True, showfliers=False,vert=True)
plt.xlabel("IMPORTANCE muscle")
plt.show()'''

In [ ]:
sum_g2g_l2l_p = sum_g2g_l2l[sum_g2g_l2l > np.quantile(sum_g2g_l2l,.98)]
sum_g2g_r2r_p = sum_g2g_r2r[sum_g2g_r2r > np.quantile(sum_g2g_r2r,.98)]

In [ ]:
del importance

In [ ]:
'''
plt.boxplot(sum_g2g_l2l_p,patch_artist=True,autorange= True, showfliers=False,vert=True)
plt.xlabel("IMPORTANCE Lung (99 percentile)")
plt.show()
plt.boxplot(sum_g2g_r2r_p,patch_artist=True,autorange = True, showfliers=False,vert=True)
plt.xlabel("IMPORTANCE Muscle (99 percentile)")
plt.show()'''

In [ ]:
'''from scipy.stats import gaussian_kde
mn,mx = np.min(sum_g2g_r2r_p), np.max(sum_g2g_r2r_p)
x = np.linspace(mn, mx, num=100)
density = gaussian_kde(sum_g2g_r2r_p)
#density.covariance_factor = lambda: .4
left_density = density(x)
plt.plot(x,left_density)
print(mn,mx)
print(x[:10])
plt.show()'''

In [ ]:
'''from scipy.stats import gaussian_kde
mn,mx = np.min(sum_g2g_l2l_p), np.max(sum_g2g_l2l_p)
x = np.linspace(mn, mx, num=100)
density = gaussian_kde(sum_g2g_l2l_p)
#density.covariance_factor = lambda: .4
left_density = density(x)
plt.plot(x,left_density)
print(mn,mx)
print(x[:10])
plt.show()'''

In [ ]:
print(len(sum_g2g_l2l_p))
print(len(sum_g2g_r2r_p))

In [ ]:
idex_l = np.argsort(sum_g2g_l2l)
idex_r = np.argsort(sum_g2g_r2r)


In [ ]:
from numpy import random
from random import sample
from sklearn import datasets

from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [ ]:
test = pd.read_csv(filepath+"exps_CommonLeft.csv")
test = test.drop(columns=['Unnamed: 0'])
left_gene_names = test.columns
test = pd.read_csv(filepath+"exps_commonRight.csv")
test = test.drop(columns=['Unnamed: 0'])
right_gene_names = test.columns


In [ ]:

net = Network(height="1200px", width="100%", bgcolor="white", font_color="black")
net.barnes_hut()
net.show_buttons(filter_=['physics'])

In [ ]:

color = []

num = -50
#right - right connecting - edge color green
for i in idex_r[num:]: #random.sample(range(len(important_indexs_right_for_right)), k=15):
  n1 = right_gene_names[i]
  
  color.append('red')  
  n2,n3,n4 = right_gene_names[importance.important_indexs_right_for_right[i]]
  net.add_nodes([n1], color=['green'], size=[5])
  net.add_nodes([n2], color=['green'], size=[5])
  net.add_nodes([n3], color=['green'], size=[5])
  net.add_nodes([n4], color=['green'], size=[5])
  net.add_edge(n1,n2,color='springgreen')
  net.add_edge(n1,n3,color='springgreen')
  net.add_edge(n1,n4,color='springgreen')

#left to left connecting - edge color blue
for i in idex_l[num:]: #random.sample(range(len(important_indexs_left_for_left)), k=15):
  n1 = left_gene_names[i]
  
  color.append('red')  
  n2,n3,n4 = left_gene_names[importance.important_indexs_left_for_left[i]]
  net.add_nodes([n1], color=['green'], size=[5])
  net.add_nodes([n2], color=['green'], size=[5])
  net.add_nodes([n3], color=['green'], size=[5])
  net.add_nodes([n4], color=['green'], size=[5])
  net.add_edge(n1,n2,color='#459ced')
  net.add_edge(n1,n3,color='#459ced')
  net.add_edge(n1,n4,color='#459ced')

for i in idex_l[num:]: #random.sample(range(len(important_indexs_right_for_left)), k=15):
  n1 = left_gene_names[i]
  
  color.append('red')  
  n2,n3,n4 = right_gene_names[importance.important_indexs_right_for_left[i]]
  net.add_nodes([n1], color=['green'], size=[5])
  net.add_nodes([n2], color=['green'], size=[5])
  net.add_nodes([n3], color=['green'], size=[5])
  net.add_nodes([n4], color=['green'], size=[5])
  net.add_edge(n1,n2,color='#e85815')
  net.add_edge(n1,n3,color='#e85815')
  net.add_edge(n1,n4,color='#e85815')

for i in idex_r[num:]: #random.sample(range(len(important_indexs_left_for_right)), k=15):
  n1 = right_gene_names[i]
  
  color.append('red')  
  n2,n3,n4 = left_gene_names[importance.important_indexs_left_for_right[i]]
  net.add_nodes([n1], color=['green'], size=[5])
  net.add_nodes([n2], color=['green'], size=[5])
  net.add_nodes([n3], color=['green'], size=[5])
  net.add_nodes([n4], color=['green'], size=[5])
  net.add_edge(n1,n2,color='#e85815')
  net.add_edge(n1,n3,color='#e85815')
  net.add_edge(n1,n4,color='#e85815')

In [ ]:
net.show("skin_blood.html")

In [ ]:
'''import cv2
cv2.imwrite('image.png', )'''

In [ ]:
'''import pickle
import numpy as np
with open('left-left.pickle', 'rb') as handle:
    gene_to_gene_left_for_left = pickle.load(handle)'''

In [ ]:
'''from colorsys import hls_to_rgb

def colorize(z):
    n,m = z.shape
    c = np.zeros((n,m,3))
    c[np.isinf(z)] = (1.0, 1.0, 1.0)
    c[np.isnan(z)] = (0.5, 0.5, 0.5)

    idx = ~(np.isinf(z) + np.isnan(z))
    A = (np.angle(z[idx]) + np.pi) / (2*np.pi)
    A = (A + 0.5) % 1.0
    B = 1.0 - 1.0/(1.0+abs(z[idx])**0.3)
    c[idx] = [hls_to_rgb(a, b, 0.8) for a,b in zip(A,B)]
    return c'''

In [ ]:
'''
n = np.array(right_importance_for_right)
plt.imshow(n/(np.max(n)-np.min(n)), cmap = 'viridis')
plt.show()'''

In [ ]:
'''test = np.sort(sum_g2g_l2l)
plt.scatter([i for i in range(len(sum_g2g_l2l))], test, s = 0.5 )
plt.show()'''

In [ ]:
import numpy as np



np.random.seed(47)
arr = np.random.rand(18000, 18000)
#print(arr)
ii = np.unravel_index(np.argpartition(arr.ravel(),-3)[-3:], arr.shape)
print(ii)

In [ ]:
del arr